In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import csv
import time

# Bulding Dataframe for all  anime 

In [2]:
url='https://www.anime-planet.com/anime/all?sort=title&order=asc'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

## 1) Scrabing anime planet website

### this data will show us name , number of episodes, rating, production year and description for all animes

In [4]:
# web scrapping for website
url='https://www.anime-planet.com/anime/all?sort=title&order=asc'

from requests import ConnectionError
with open('Data.csv', 'a+', encoding="utf-8") as f:
    fieldnames=['Name','TotalEpisodes', 'Rating','ProdYear','AnimeType','DataID','DataType','EpsData','EpsType','HtmlREF','tag','description','Src']
    writer = csv.writer(f)   
    writer.writerow(fieldnames)
    while True:
        while True:
            # get the data from anime planet.
            try:
                res = requests.get(url)
                break
            except ConnectionError:
                time.sleep(3)
                continue 
        if res.status_code != 204:

            soup = BeautifulSoup(res.text, 'html.parser')

            for sp in soup.find('ul', attrs={'class':'cardDeck cardGrid'}).find_all('li'):
                # 1)  get the anime name
                Name=sp.find('h3', {'class':'cardName'}).get_text()
                # 2)  get the tags of the anime
                def tags():
                    try:
                        listTag = []
                        for tag in BeautifulSoup(sp.find('a').get('title')).body.find('div', {'class':'tags'}).find_all('li'):
                            listTag.append(tag.get_text())
                        return listTag
                    except (AttributeError):
                        return(np.nan)
                tag =tags()
                # 3)  get description
                description=BeautifulSoup(sp.find('a').get('title')).body.find('p').get_text()
                # 4)  get the source if exist
                def Source():
                    try:
                        return BeautifulSoup(sp.find('a').get('title')).body.find_all('p')[1].get_text().split(':')[1].strip()
                    except(IndexError):
                        return np.nan
                Src = Source()
                # 5)  anime rating
                def rating():
                    try:
                        return float(BeautifulSoup(sp.find('a').get('title')).body.ul.find('div', attrs={'class':'ttRating'}).get_text())
                    except:
                        return np.nan
                Rating=rating()
                # 6)  anime production year
                def year():
                    try:
                        return BeautifulSoup(sp.find('a').get('title')).body.ul.find('li', attrs={'class':'iconYear'}).get_text()
                    except:
                        return np.nan

                ProdYear=year()
                # 7 ) anime type
                AnimeType=BeautifulSoup(sp.find('a').get('title')).body.ul.find('li', attrs={'class':'type'}).get_text().split()[0]
                # 8)  data id 
                DataID=sp.get('data-id')
                # 9)  data type
                DataType=sp.get('data-type')
                # 10) data of episodes
                EpsData=sp.get('data-episodes')
                # 11) episode type
                EpsType=sp.get('data-episode-type')
                # 12) total episodes
                TotalEpisodes=sp.get('data-total-episodes')
                # 13) html refrence
                HtmlREF= sp.find('a').get('href')
                writer.writerow([Name,TotalEpisodes, Rating,ProdYear,AnimeType,DataID,DataType,EpsData,EpsType,HtmlREF,tag,description,Src])

            # get the url of the nex page

            url= url[:url.rfind('/')]+'/'+soup.find('li',attrs={'class':'next'}).find('a').get('href').split('/')[2]


AttributeError: 'NoneType' object has no attribute 'get'

In [3]:
df = pd.read_csv('Data.csv')


## Here is how the Dataframe looks like:

In [4]:
df.loc[df.Src.isnull()].loc[df.AnimeType == 'Music']

,Name,TotalEpisodes,Rating,ProdYear,AnimeType,DataID,DataType,EpsData,EpsType,HtmlREF,tag,description,Src
8,1000say: HANE,1,2.1,2011,Music,10928,anime,NaN,episodes,/anime/1000say-hane,"['Fantasy', 'CG Animation']",A young girl wishes that she could grow wings ...,NaN
26,123,1,2.8,2004,Music,10384,anime,NaN,episodes,/anime/123,"['Family Friendly', 'Minna no Uta', 'Music']",This entry currently doesn't have a synopsis. ...,NaN
37,171 Eru: Majo Le Septet,1,2.7,2015,Music,12017,anime,NaN,episodes,/anime/171-eru-majo-le-septet,['Vocaloid'],This entry currently doesn't have a synopsis. ...,NaN
40,1980YEN: Takoyaki Story,1,2.7,2018,Music,12700,anime,NaN,episodes,/anime/1980yen-takoyaki-story,"['Abstract', 'Food and Beverage', 'Nudity', 'S...",The official music video for Takoyaki Story fr...,NaN
42,2002-nen Baachan Ondo,1,2.8,2002,Music,13576,anime,NaN,episodes,/anime/2002-nen-baachan-ondo,"['Sports', 'Minna no Uta', 'Soccer']",This entry currently doesn't have a synopsis. ...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18376,Zutomayo: Obenkyou Shitoiteyo,1,3.5,2020,Music,15030,anime,NaN,episodes,/anime/zutomayo-obenkyou-shitoiteyo,NaN,The official music video for Obenkyou Shitoite...,NaN
18377,Zutomayo: Saturn,1,3.4,2020,Music,16547,anime,NaN,episodes,/anime/zutomayo-saturn,NaN,"The official music video for the song Saturn, ...",NaN
18378,Zutomayo: Seigi,1,3.3,2019,Music,15687,anime,NaN,episodes,/anime/zutomayo-seigi,NaN,The official music video for Seigi from the al...,NaN
18379,Zutomayo: Tadashiku Narenai,1,3.3,2020,Music,16117,anime,NaN,episodes,/anime/zutomayo-tadashiku-narenai,['CG Animation'],The official music video for Tadashiku Narenai...,NaN


In [8]:
df.EpsType.unique()

array(['episodes'], dtype=object)

In [9]:
# delete useless data
df.drop(['EpsData', 'EpsType'], axis=1, inplace=True)

In [10]:
# write the modified datato a new csv file 
df.to_csv('DataModified.csv', index=False)

In [11]:
# turn str to list
def StrToList(x):
    try:
        return eval(x)
    except:
        return x
# df.tag = df['tag'].apply(StrToList)
# d.tag.value_counts()
type(df.tag[0])


str

In [12]:
# TODO get the main genre
r =requests.get('https://www.anime-planet.com/anime/all')
s = BeautifulSoup(r.text, 'html.parser')

MG =[]
for g in s.find('div', {'id':'multipletags'}).ul.find_all('li', attrs={'data-class': 'filter-tooltip'}):
    MG.append(g.find('a').get_text())
MG

['Action',
 'Adventure',
 'BL',
 'Comedy',
 'Drama',
 'Ecchi',
 'Fantasy',
 'GL',
 'Harem',
 'Horror',
 'Josei',
 'Magical Girl',
 'Mecha',
 'Mystery',
 'Reverse Harem',
 'Romance',
 'Sci Fi',
 'Seinen',
 'Shoujo',
 'Shoujo-ai',
 'Shounen',
 'Shounen-ai',
 'Slice of Life',
 'Sports',
 'Yaoi',
 'Yuri']